In [1]:
#On va importer les packages 
!pip install wordcloud 
!pip install nltk
!pip install pattern

import pandas as pd
import re
import os

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('french'))
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

import matplotlib.pyplot as plt
from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.1/511.1 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 29.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 43.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [27 lines of output]
      /bin/sh: 1: pkg-config: not found
      /bin/sh: 1: pkg-config: not found
      /bin/sh: 1: pkg-config: not found
      Trying pkg-config --exists mysqlclient
      Command 'pkg-config --exists mysqlclient' returned non-zero exit status 127.
      Trying pkg-config --exists mariadb
      Command 'pkg-config --exists mariadb' returned no

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /home/onyxia/nltk_data...


In [2]:
#Récupérations données 
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers le répertoire de données
fichier = os.path.join(base_dir, 'data', 'base_def.csv')

df = pd.read_csv(fichier)

In [3]:
# Travail des stop words
mots_fr = [
    'le', 'la', 'les', 'de', 'un', 'à', 'être', 'suis', 'es', 'est', 'sont', 'été', 'étais', 'était', 'étaient', 'serai', 'seras', 'sera', 'seront',
    'et', 'en', 'avoir', 'ai', 'as', 'a', 'ont', 'eu', 'aurai', 'auras', 'aura', 'auront', 'avais', 'avait', 'avaient', 'que', 'pour',
    'dans', 'ce', 'il', 'qui', 'ne', 'en', 'sur', 'se', 'pas', 'plus',
    'par', 'je', 'avec', 'tout', 'faire', 'fais', 'fait', 'font', 'son', 'mettre', 'autre', 'on', 'mais',
    'nous', 'comme', 'ou', 'si', 'leur', 'y', 'dire', 'dis', 'dit', 'disent' 'elle', 'voir', 'devoir',
    'deux', 'même', 'pendant', 'aussi', 'vouloir', 'grand', 'mon', 'nouveau', 'aller',
    'venir', 'ceux', 'faire', 'quelque', 'trouver', 'donner', 'donne', 'donnes', 'donnent', 'aussi', 'autre',
    'tous', 'vieux', 'bon', 'voir', 'moins', 'trois', 'avant', 'sa', 'faire', 'contre',
    'abord', 'sous', 'ou', 'apporter', 'grand', 'ainsi', 'long', 'très', 'tout', 'avoir',
    'beau', 'chaque', 'peu', 'quoi', 'encore', 'aller', 'montrer', 'semaine', 'ainsi',
    'nuit', 'aussi', 'bien', 'deuxième', 'moins', 'tout', 'avoir', 'peu', 'nuit', 'ça',
    'mon', 'ma', 'mes', 'ton', 'ta', 'tes', 'son', 'sa', 'ses', 'notre', 'notre', 'nos', 'votre', 'votre', 'vos', 'leur', 'leur', 'leurs',
    'non', '’', '.', ',', ':', 'est', 'des', "l'", "d'", "une", "d", "»", "«", "du", "qu", "au", "n", "s", "elle", ")", "(", "aux", 'c', 'cette', 'un', 'ces','?', 'il', 'ils', 'j', "l", "-", "vous", "entre",
    "[", "]", "elles", '”', '“', "dont", "cela", ";", "m", "selon", "!", '–', "en", "autre", "ici", "ce", "ça", "cela", "ceci", "car", "ou", "où", "ni", "mais", "et", "donc", "parce", "que", "quand", "comment", "qui", "quoi",
    "à", "de", "pour", "sans", "sur"
]
# Concaténer tous les textes de la colonne 'contenu' en une seule chaîne
all_text = ' '.join(df['contenu'])

# Fonction pour enlever les mots de la liste mots_fr
def remove_words(text):
    words = word_tokenize(text.lower())  # normaliser la casse ici
    filtered_words = [word for word in words if word not in mots_fr]
    return ' '.join(filtered_words)
    
# Appliquer la fonction remove_words
df['contenu'] = df['contenu'].apply(lambda x: remove_words(x))

# Fonction pour normaliser les mots au singulier [va falloir singulariser autrement]
def singularize_word(word):
    # Retire le 's' à la fin du mot si présent et si la version sans 's' existe déjà strictement
    if word.lower().endswith('s') and word[:-1].lower() in set(all_text.split()):
        return word[:-1]
    else:
        return word
        
df['contenu'] = df['contenu'].apply(lambda x: ' '.join([singularize_word(word) for word in x.split()]))

all_text2 = ' '.join(df['contenu'])

# Fonction pour obtenir les mots les plus représentés
def get_most_common_words(text):
    words = word_tokenize(text.lower())
    freq_dist = FreqDist(words)
    return freq_dist.most_common()

# Appliquer la fonction à l'ensemble du texte
most_common_words = get_most_common_words(all_text2)
most_common_words

[('femme', 2565),
 ('homme', 1328),
 ('genre', 935),
 ('sexe', 887),
 ('personne', 786),
 ('droit', 564),
 ('enfant', 564),
 ('féministe', 442),
 ('toute', 412),
 ('violence', 403),
 ('sexuelle', 379),
 ('identité', 355),
 ('trans', 350),
 ('autre', 345),
 ('jeune', 339),
 ('peut', 332),
 ('question', 315),
 ('monde', 304),
 ('corps', 301),
 ('fille', 301),
 ('an', 297),
 ('me', 284),
 ('politique', 279),
 ('vie', 274),
 ('année', 274),
 ('transgenre', 264),
 ('alors', 261),
 ('tant', 256),
 ('mouvement', 250),
 ('façon', 247),
 ('chose', 238),
 ('soit', 228),
 ('lui', 227),
 ('hui', 225),
 ('idéologie', 225),
 ('savoir', 221),
 ('sexuel', 221),
 ('loi', 220),
 ('aujourd', 216),
 ('terme', 213),
 ('pouvoir', 207),
 ('cas', 206),
 ('groupe', 204),
 ('moi', 202),
 ('toujours', 202),
 ('également', 200),
 ('livre', 199),
 ('foi', 198),
 ('travail', 197),
 ('après', 196),
 ('femelle', 193),
 ('lesbienne', 191),
 ('humain', 190),
 ('jamais', 187),
 ('donnée', 184),
 ('réalité', 183),
 ('peu

In [6]:
mots_fr2 = ["m.", '—', '``', "autre", "peut", "peuvent", "ont", "a", "suis", "es", "sont", "sommes", "êtes", "me", "te", "se", "tant", "alors", "toute", "tout", "tous", "toutes", "façon", "chose", "aujourd", "hui",
           "façon", "soit", "lui", "moi"]

def remove_words2(text):
    words = word_tokenize(text.lower())  # normaliser la casse ici
    filtered_words = [word for word in words if word not in mots_fr2]
    return ' '.join(filtered_words)
    
# Appliquer la fonction remove_words
df['contenu'] = df['contenu'].apply(lambda x: remove_words2(x))

all_text2 = ' '.join(df['contenu'])

most_common_words = get_most_common_words(all_text2)
most_common_words[:50]

[('femme', 2565),
 ('homme', 1328),
 ('genre', 935),
 ('sexe', 887),
 ('personne', 786),
 ('droit', 564),
 ('enfant', 564),
 ('féministe', 442),
 ('violence', 403),
 ('sexuelle', 379),
 ('identité', 355),
 ('trans', 350),
 ('jeune', 339),
 ('question', 315),
 ('monde', 304),
 ('corps', 301),
 ('fille', 301),
 ('an', 297),
 ('politique', 279),
 ('vie', 274),
 ('année', 274),
 ('transgenre', 264),
 ('mouvement', 250),
 ('idéologie', 225),
 ('savoir', 221),
 ('sexuel', 221),
 ('loi', 220),
 ('terme', 213),
 ('pouvoir', 207),
 ('cas', 206),
 ('groupe', 204),
 ('toujours', 202),
 ('également', 200),
 ('livre', 199),
 ('foi', 198),
 ('travail', 197),
 ('après', 196),
 ('femelle', 193),
 ('lesbienne', 191),
 ('humain', 190),
 ('jamais', 187),
 ('donnée', 184),
 ('réalité', 183),
 ('rapport', 181),
 ('partie', 179),
 ('critique', 179),
 ('société', 178),
 ('masculin', 178),
 ('féminisme', 176),
 ('organisation', 175)]

In [7]:
# Obtenez le chemin du répertoire parent du répertoire src
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers le répertoire de données
chemin_data = os.path.join(base_dir, 'data')

# Enregistrez le DataFrame dans un fichier CSV dans le répertoire data
df.to_csv(os.path.join(chemin_data, 'base_clean.csv'), index=False)

In [8]:
#Récupérations données 
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers le répertoire de données
fichier = os.path.join(base_dir, 'data', 'base_clean.csv')

df = pd.read_csv(fichier)